In [1]:
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
from tensorflow import keras
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
import matplotlib as mpl
import matplotlib.pyplot as plt

#numpy配列を省略しないようにする
np.set_printoptions(threshold=np.inf)
#pandasを省略しないようにする
pd.set_option('display.max_columns', 500) # 列
pd.set_option('display.max_rows', 500)  # 行

In [2]:
# 正規化関数
def mmscaler(data):
    mm = preprocessing.MinMaxScaler()  # 正規化エンコード、デコード
    dat = mm.fit_transform(data)
    return dat

In [3]:
# トレーニングデータの作成
def create_data(read_dir, save_dir):
    df = pd.read_csv(read_dir, encoding='shift_jis')
    #カラムを削除
    df = df.dropna()  # NaNを削除
    df.drop(labels='日付(ローソク足)', axis=1, inplace=True) # ろうそく足データの日付
    
    df['日付'] = pd.to_datetime(df['日付'], format='%Y-%m-%d-%A %H:%M:%S')  # 日付カラムを日付型に変換
    df['日付(hour)'] = df['日付'].dt.hour  # hourをデータに追加
    df['日付(minute)'] = df['日付'].dt.minute  # minuteをデータに追加
    df['日付(weekday)'] = df['日付'].dt.dayofweek  # minuteをデータに追加
    # df['日付']カラムが [datetime64[ns]]型になっていて扱いづらいので最後に削除する
    #df.drop(labels='日付', axis=1, inplace=True)
    
    date = df['日付'] # 代入して変数を避難させる
    
    df = df.loc[:, '2':'日付(weekday)'].astype('float32')  # データ型をfloatに変換して代入
    df.insert(0, '日付', date)  # 日付を1列目に代入
    #df.to_csv(save_dir , encoding='shift_jis', index=False)
    return df

In [4]:
# n時間後の時刻とレートを計算
def UNIX_RATE_conversion(df, time, name):
    # UNIX時間に変更
    df['日付UNIX'] = pd.to_datetime(df['日付']).dt.tz_localize('Asia/Tokyo')
    df['日付UNIX'] = df['日付UNIX'].astype('int64') // 10**9
    
    df['日付UNIX' + name] = df['日付UNIX'].shift(time) # 1行上にずらして5分後の値にしている
    df['日付UNIX_計算後' + name] = df['日付UNIX' + name] - df['日付UNIX'] # -分後の値か表した変数
    df['RATE' + name] = df['bid_close'].shift(time)

    return df  # n分後のUNIX時間、レートの計算結果を反映している

In [5]:
# 変数宣言
read_dir = "./shape/GBP_JPY_X_train_data.csv"
save_dir =  "./model/GBPJPY_5M.csv"
mm = preprocessing.MinMaxScaler()  # 正規化エンコード、デコード

In [6]:
# トレーニングデータの作成
df = create_data(read_dir, save_dir)

# 5分後の時刻とレートを計算
after_5min = -1 # shift関数のスライド数 (1行上にずらして5分後の値にしている)
name = '5min'
df = UNIX_RATE_conversion(df, after_5min,name)



# 日付チェック
df = df[(df['日付UNIX_計算後' + name] < 320) & (270 < df['日付UNIX_計算後' + name])] # 日付の差が270秒から320秒の間のみ抽出

df = df[2::4]  # 2行目から4行飛ばしで読み込む(oandaの会員がレギュラーのため)
df.to_csv("./model/5分後予測のGBPJPYデータ.csv", encoding='shift_jis', index=False)

# 4時間後の時刻とレートを計算
after_4h = -47
name = '4h'
df = UNIX_RATE_conversion(df, after_4h, name)

In [7]:
def model_create():
    #n_inputs = len(X_train.columns)  # 入力数
    model = keras.models.Sequential()
   # model.add(keras.layers.Dense(1200, activation='sigmoid'))
   # model.add(keras.layers.Dense(300, activation='sigmoid'))
   # model.add(keras.layers.Dense(200, activation='linear'))
    model.add(keras.layers.LSTM(1, activation='linear',
                  #recurrent_activation='linear',
                  kernel_initializer='glorot_normal',
                   recurrent_initializer='orthogonal',
                               batch_input_shape=(None, n_inputs, 1)))
    #model.add(keras.layers.Dense(1200, activation='linear'))
    #model.add(keras.layers.Dense(100, activation='linear'))
    #model.add(keras.layers.Dense(50, activation='linear'))
    #model.add(keras.layers.Dense(7, activation='linear'))
    #model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    optimizer = keras.optimizers.Adam(lr=0.1)  # オプティマイザ
    loss_fn = keras.losses.mse  # 損失関数
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['mae'])  # コンパイル
    #model.summary()
    return model

In [8]:
# 入力したインプットデータをdropna、正規化、形状を整える関数
def create_traindata(df, X_train, y_train):


    X_train = X_train.dropna()  # NaNを削除
    y_train = y_train.dropna()  # NaNを削除
    print('入力数:', X_train.columns)
    row_count = len(X_train)  # 行数を取得(形状の作成に使用)
    column_count = len(X_train.columns)  # 列数を取得(形状の作成に使用)
    print('行数:', row_count)
    print('列数:', column_count)
    print('整形前の形状(特徴量):', X_train.shape)
    X_train = mm.fit_transform(X_train)  # 正規化
    X_train = np.array(X_train).reshape(row_count, column_count, -1)  # 特徴量の形状(3次元)
    y_train = np.array(y_train).reshape(row_count,1)  # 特徴量の形状
    y_train = mm.fit_transform(y_train)  # 正規化

    print('形状(特徴量):', X_train.shape)
    print('形状(目的変数):', y_train.shape)
    return X_train, y_train

In [9]:
# モデルの学習関数
def learn_model(X_train, y_train):
    model = model_create()  # モデル作成
    earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
    # 学習 LSTM_GBPJPY
    print('モデルの学習を開始します')
    history_model = model.fit(X_train, y_train, batch_size=1,epochs=25,validation_split=0.05,
                                                #callbacks=[tensorboard,earlystopping]
                                               )
    return model

In [10]:
# 予測関数
def predict(model, X_train, y_train):
    y_pred = model(X_train)
    y_pred = mm.inverse_transform(y_pred)  # 予測結果の正規化をデコード
    y_train = mm.inverse_transform(y_train)  # 実際の結果の正規化をデコード
    return y_pred, y_train

In [11]:
# プロット関数
def plot_result(y_pred, y_train):
    flg = plt.figure()
    fig = plt.figure(figsize=(6, 4), dpi=72, linewidth=10)
    ax = fig.add_subplot(111, xlabel='number', ylabel='price',title='pred_act')  # グラフ作成

    ax.plot(y_pred, label='predict')  # 予測
    ax.plot(y_train, label='actual')  # 実測
    plt.legend()

In [ ]:
#**********5分後予測**********学習のinput情報(特徴量、答え)

X_train = df.loc[:, '2':'日付(weekday)']  # 全行 , 列名称(始まり):列名称(終わり)
y_train = df.loc[:, 'RATE5min']  # 全行 , 最終列
n_inputs = len(X_train.columns)  # 入力数
X_train, y_train = create_traindata(df, X_train, y_train)  # 入力したインプットデータをdropna、正規化、形状を整える
model = learn_model(X_train, y_train)  # モデルの学習
y_pred, y_train = predict(model, X_train, y_train) # 予測
plot_result(y_pred, y_train)  # プロット

入力数: Index(['2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       ...
       '2433', 'complete', 'volume', 'bid_open', 'bid_high', 'bid_low',
       'bid_close', '日付(hour)', '日付(minute)', '日付(weekday)'],
      dtype='object', length=2441)
行数: 471
列数: 2441
整形前の形状(特徴量): (471, 2441)
形状(特徴量): (471, 2441, 1)
形状(目的変数): (471, 1)
モデルの学習を開始します
Epoch 1/25
447/447 [==============================] - 132s 294ms/step - loss: 0.0657 - mae: 0.2041 - val_loss: 0.0031 - val_mae: 0.0450
Epoch 2/25
447/447 [==============================] - 131s 294ms/step - loss: 0.0562 - mae: 0.1659 - val_loss: 0.3443 - val_mae: 0.5838
Epoch 3/25
447/447 [==============================] - 127s 284ms/step - loss: 0.4531 - mae: 0.6267 - val_loss: 0.7387 - val_mae: 0.8578
Epoch 4/25
447/447 [==============================] - 128s 287ms/step - loss: 0.4416 - mae: 0.6144 - val_loss: 0.7253 - val_mae: 0.8499
Epoch 5/25
447/447 [==============================] - 132s 296ms/step - loss: 0.4416 - mae: 0.6111 - val_loss: 0.7

In [ ]:
y_pred

In [ ]:
y_train

In [ ]:
y_train - y_pred

In [ ]:
b = y_train.tolist()

In [ ]:
#model = keras.models.load_model('model.hdf5')  # モデルを読込み
#model.load_weights('param.hdf5')  # 重みを読込み

In [ ]:
model.save('model.hdf5')  # モデルを保存
model.save_weights('param.hdf5')  # 重みを保存